the purpose of this file is to create a csv file for each exome file, with the number of reads/not reads in each position

In [1]:
import pandas as pd
import os
from IPython.display import clear_output
from functools import reduce

In [2]:
FILES_LIST = ["/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ab_1..534.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ad_1..547.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ae_1..524.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ab_1..2641.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_aa_1..536.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_ab_1..532.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_ac_1..521.g.vcf.gz",]
CURR_FILE = 0
BATCH_SIZE = 10000
BLANK_CELL_PREFIX = "\./\."
BLANK_VAL = "."

In [ ]:
count = 0
dfs = []

In [ ]:

df = pd.read_csv(FILES_LIST[CURR_FILE],compression='gzip',skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)),nrows=BATCH_SIZE,sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
while len(df) > 10:
    clear_output(wait=True)
    print(count)
    df = pd.read_csv(FILES_LIST[CURR_FILE],compression='gzip',skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)),nrows=BATCH_SIZE,sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
    df["blank_sum"] = df.eq(BLANK_VAL).sum(axis=1)
    dfs.append(df[["#CHROM",	"POS",	"ID",	"REF",	"ALT",	"QUAL",	"FILTER",	"INFO",	"FORMAT","blank_sum"]])
    count += 1


693


In [22]:
len(dfs)

545

## Map exome files locally

In [ ]:
JOINED_PATH = "/sci/archive/michall/roeizucker/col_hackaton/joined_multiple_files_2"
SAVE_FILE_COUNT_SPACE = 20
# df = pd.read_csv(FILES_LIST[CURR_FILE],compression='gzip',skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)),nrows=BATCH_SIZE,sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
# curr_file = CURR_FILE
# dfs = []
# while len(df) > 10:
#     clear_output(wait=True)
#     print(count)
#     df = pd.read_csv(FILES_LIST[curr_file],compression='gzip',skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)),nrows=BATCH_SIZE,sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
#     df["blank_sum"] = df.eq(BLANK_VAL).sum(axis=1)
#     dfs.append(df[["#CHROM",	"POS",	"ID",	"REF",	"ALT",	"QUAL",	"FILTER",	"INFO",	"FORMAT","blank_sum"]])
#     if count != 0 and count % SAVE_FILE_COUNT_SPACE == 0:
#         new_df = pd.concat(dfs, ignore_index=True)
#         dfs = []
#         new_df.to_csv(os.path.join(JOINED_PATH,f"file_{curr_file}_batch{count // SAVE_FILE_COUNT_SPACE}.csv"))
#     count += 1
# if (count -1) % 20 != 0:
#     new_df.to_csv(os.path.join(JOINED_PATH,f"file_{curr_file}_batch{(count // SAVE_FILE_COUNT_SPACE) + 1}.csv"))

# iter_csv = pd.read_csv(FILES_LIST[curr_file],compression='gzip', iterator=True, chunksize=100,skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)))
for curr_file in range(len(FILES_LIST)):
    if os.path.exists(os.path.join(JOINED_PATH,f"file_{curr_file + 1}_batch1.csv")):
        print("done with",curr_file)
        continue
    iter_csv = pd.read_csv(FILES_LIST[curr_file],compression='gzip', iterator=True, chunksize=BATCH_SIZE,skiprows=list(range(3381)),sep="\t")
    count = 0
    dfs = []
    for df in iter_csv:
        clear_output(wait=True)
        print("count:",count, "file:",curr_file)
        if os.path.exists(os.path.join(JOINED_PATH,f"file_{curr_file}_batch{(count+SAVE_FILE_COUNT_SPACE) // SAVE_FILE_COUNT_SPACE}.csv")):
            print("done")
            count+=1
            continue
        df = df.replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
        df["blank_sum"] = df.eq(BLANK_VAL).sum(axis=1)
        dfs.append(df[["#CHROM",	"POS",	"ID",	"REF",	"ALT",	"QUAL",	"FILTER",	"INFO",	"FORMAT","blank_sum"]])
        if count != 0 and count % SAVE_FILE_COUNT_SPACE == 0:
            new_df = pd.concat(dfs, ignore_index=True)
            dfs = []
            new_df.to_csv(os.path.join(JOINED_PATH,f"file_{curr_file}_batch{count // SAVE_FILE_COUNT_SPACE}.csv"))
        count+=1
    if (count -1) % 20 != 0:
        new_df.to_csv(os.path.join(JOINED_PATH,f"file_{curr_file}_batch{(count // SAVE_FILE_COUNT_SPACE) + 1}.csv"))
    print("done with",curr_file)

count: 5127 file: 5


## combine mapped files

In [56]:
for curr_file in range(len(FILES_LIST)):
    num = 1
    dfs = []
    file_name= os.path.join(JOINED_PATH,f"file_{curr_file}_batch{num}.csv")
    
    while os.path.exists(file_name):
        clear_output(wait=True)
        print("num:",num, "file:",curr_file)
        df = pd.read_csv(file_name)
        dfs.append(df)
        num +=1
        file_name= os.path.join(JOINED_PATH,f"file_{curr_file}_batch{num}.csv")
    combined_df = pd.concat(dfs,ignore_index=True)
    combined_df.to_csv(os.path.join(JOINED_PATH,f"file_{curr_file}.csv"))

num: 260 file: 7


## combine the combined files

In [ ]:
dfs = []
for i in range(len(FILES_LIST)):
# for i in range(2):
    print(i)
    df = pd.read_csv(os.path.join(JOINED_PATH,f"file_{i}.csv"))
    df["blank_sum_" + str(i)] = df["blank_sum"]
    df = df.drop(columns=["blank_sum"])
#     add length
    dfs.append(df)
df_final = reduce(lambda x,y: pd.merge(x,y, on=["#CHROM","POS"], how='outer'), dfs)

df_final["blank_sum_all"] = 0
for i in range(len(dfs)):
    temp_df = pd.read_csv(FILES_LIST[i],compression='gzip',skiprows=list(range(3381)),nrows=2,sep="\t" )
    n_cols = len(temp_df.columns) - 9
    df_final["blank_sum_all"] = df_final["blank_sum_all"] + df_final["blank_sum_" + str(i)].fillna(n_cols)
df_final[["#CHROM","POS","blank_sum_all"]].to_csv(os.path.join(JOINED_PATH,"combined_all.csv"))

0
1
2
3
4
5
6
7


/cs/labs/michall/roeizucker/fml_virt_env/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x', 'Unnamed: 0.1_x', 'FORMAT_x', 'QUAL_x', 'REF_x', 'ID_x', 'FILTER_x', 'ALT_x', 'INFO_x'} in the result is deprecated and will raise a MergeError in a future version.
  # Remove the CWD from sys.path while we load stuff.


,Unnamed: 0_x,Unnamed: 0.1_x,#CHROM,POS,ID_x,REF_x,ALT_x,QUAL_x,FILTER_x,INFO_x,...,Unnamed: 0_y,Unnamed: 0.1_y,ID_y,REF_y,ALT_y,QUAL_y,FILTER_y,INFO_y,FORMAT_y,blank_sum_1
0,0.0,0.0,chr1,68991,.,A,<*>,0.0,.,END=69172,...,0.0,0.0,.,A,<*>,0.0,.,END=69990,GT:GQ:MIN_DP:MED_DP:PL,2.0
1,1.0,1.0,chr1,68992,.,T,<*>,0.0,.,END=69067,...,1.0,1.0,.,T,<*>,0.0,.,END=69013,GT:GQ:MIN_DP:MED_DP:PL,535.0
2,2.0,2.0,chr1,68994,.,T,<*>,0.0,.,END=69051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,3.0,chr1,68996,.,A,<*>,0.0,.,END=68997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,chr1,68998,.,T,<*>,0.0,.,END=68998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55855739,NaN,NaN,chrY,7085929,NaN,NaN,NaN,NaN,NaN,NaN,...,51209684.0,199684.0,.,C,<*>,0.0,.,END=7085934,GT:GQ:MIN_DP:MED_DP:PL,534.0
55855740,NaN,NaN,chrY,7085939,NaN,NaN,NaN,NaN,NaN,NaN,...,51009693.0,199693.0,.,G,<*>,0.0,.,END=7085939,GT:GQ:MIN_DP:MED_DP:PL,535.0
55855741,NaN,NaN,chrY,7085939,NaN,NaN,NaN,NaN,NaN,NaN,...,51209693.0,199693.0,.,G,<*>,0.0,.,END=7085939,GT:GQ:MIN_DP:MED_DP:PL,535.0
55855742,NaN,NaN,chrY,7085940,NaN,NaN,NaN,NaN,NaN,NaN,...,51009694.0,199694.0,.,C,<*>,0.0,.,END=7085978,GT:GQ:MIN_DP:MED_DP:PL,535.0


,#CHROM,POS,blank_sum_all
0,chr1,68991,4.0
1,chr1,68992,1050.0
2,chr1,68994,1050.0
3,chr1,68996,1050.0
4,chr1,68998,1049.0
...,...,...,...
55855739,chrY,7085929,1049.0
55855740,chrY,7085939,1050.0
55855741,chrY,7085939,1050.0
55855742,chrY,7085940,1050.0


## script

In [55]:
res = int(os.popen("zcat /sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz | head -n 12 | wc -l").read())

12

In [4]:
MASTER_FILE_PATH = "/sci/nosnap/michall/roeizucker/col_hackaton/master.sh"
MEDIATOR_FILE_PATH = "/sci/nosnap/michall/roeizucker/col_hackaton/mediator.py"

In [32]:
# part1 - in jupyter

# find the file length, 
# FILE_LENGTH = int(os.popen("zcat /sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz | wc -l").read())
FILE_LENGTH = 51049008
# FILE = "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz"
FILE= "/sci/archive/michall/roeizucker/col_hackaton/bbatch_sample_LDL_ids.aa_ad_1..547.g.vcf"
# part2 - master script
# find the number of instances and my instance.
SKIP = 3381
BATCH_SIZE = 10000
# NUMBER_OF_INSTANCES = recieved -1

NUMBER_OF_INSTANCES = 200 -1
MY_INSTANCE = 199

# find my part- which rows do I need to read
my_start = int(SKIP + (FILE_LENGTH - SKIP) * (MY_INSTANCE / (NUMBER_OF_INSTANCES + 1))) + 1
my_end = int(SKIP + (FILE_LENGTH - SKIP) * ((MY_INSTANCE+1) / (NUMBER_OF_INSTANCES + 1)))
count = 0
# df = pd.read_csv(FILES_LIST[CURR_FILE],compression='gzip',skiprows=list(range(SKIP)) + list(range(my_start,count*BATCH_SIZE + my_start)),nrows=BATCH_SIZE,sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
dfs = []
while count*BATCH_SIZE + my_start < my_end:
    print(count)
    curr = count*BATCH_SIZE + my_start
#     df = pd.read_csv(FILE,compression='gzip',skiprows=list(range(SKIP)) + list(range(SKIP + 1,count*BATCH_SIZE + my_start)),nrows=min(BATCH_SIZE,my_end - curr),sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
    df = pd.read_csv(FILE,skiprows=list(range(SKIP)) + list(range(SKIP + 1,count*BATCH_SIZE + my_start)),nrows=min(BATCH_SIZE,my_end - curr),sep="\t").replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)

    df["blank_sum"] = df.eq(BLANK_VAL).sum(axis=1)
    dfs.append(df[["#CHROM",	"POS",	"ID",	"REF",	"ALT",	"QUAL",	"FILTER",	"INFO",	"FORMAT","blank_sum"]])
    count += 1

# read these rows in jumps of 100,000 and append to list
# combine this list into csv

0


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [25]:
dfs[-1]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,blank_sum
0,chr1,8361001,.,G,<*>,0.0,.,END=8361001,GT:GQ:MIN_DP:MED_DP:PL,516
1,chr1,8361002,.,C,<*>,0.0,.,END=8361002,GT:GQ:MIN_DP:MED_DP:PL,515
2,chr1,8361003,.,T,<*>,0.0,.,END=8361031,GT:GQ:MIN_DP:MED_DP:PL,516
3,chr1,8361004,.,G,<*>,0.0,.,END=8361014,GT:GQ:MIN_DP:MED_DP:PL,515
4,chr1,8361005,.,G,<*>,0.0,.,END=8361005,GT:GQ:MIN_DP:MED_DP:PL,515
5,chr1,8361006,.,C,<*>,0.0,.,END=8361006,GT:GQ:MIN_DP:MED_DP:PL,515
6,chr1,8361007,.,C,<*>,0.0,.,END=8361007,GT:GQ:MIN_DP:MED_DP:PL,514
7,chr1,8361008,.,C,<*>,0.0,.,END=8361035,GT:GQ:MIN_DP:MED_DP:PL,515
8,chr1,8361009,.,G,"<*>,A",46.5,PASS,END=8361009,GT:GQ:MIN_DP:MED_DP:PL:DP:AD:VAF,514
9,chr1,8361010,.,C,<*>,0.0,.,END=8361023,GT:GQ:MIN_DP:MED_DP:PL,515


In [17]:

def write_master_script(master_script_path,mediator_script_path):
    with open(master_script_path, "w") as master_file:
        master_file.write(f'''
FILE_LENGTH={FILE_LENGTH}
python {mediator_script_path} $SLURM_ARRAY_TASK_ID $SLURM_ARRAY_TASK_COUNT $FILE_LENGTH
''')
    !chmod 744 {master_script_path}
write_master_script(MASTER_FILE_PATH,MEDIATOR_FILE_PATH)

In [21]:
def write_mediator_script(mediator_script_path):
    with open(mediator_script_path, "w") as mediator_file:
        mediator_file.write(f'''import sys
print(sys.argv)
''')
write_mediator_script(MEDIATOR_FILE_PATH)

In [19]:
print(f'''how to run:
sbatch --array=0-5 --mem={1 + 1}g -c{2} --time=5-0 --requeue --killable --wrap="{MASTER_FILE_PATH}"''')

how to run:
sbatch --array=0-5 --mem=2g -c2 --time=5-0 --requeue --killable --wrap="/sci/nosnap/michall/roeizucker/col_hackaton/master.sh"
